# 云台控制和 LED 灯控制
## 云台控制
### 基础介绍

带有云台的产品上含有两个舵机，分别为水平舵机和倾斜舵机。水平舵机控制云台水平平面的转动，转动范围 ±180°（总范围360°）；倾斜舵机控制云台竖直平面的转动，转动范围 -45°~90°（总范围 135°）。

对于没有配有云台的产品，也可以自行在 RaspRover 上扩展云台。

In [ ]:
# 导入用于控制底盘的库
from base_ctrl import BaseController

# 用于检测树莓派的函数
def is_raspberry_pi5():
    with open('/proc/cpuinfo', 'r') as file:
        for line in file:
            if 'Model' in line:
                if 'Raspberry Pi 5' in line:
                    return True
                else:
                    return False

# 根据树莓派的型号来确定 GPIO 串口设备名称
if is_raspberry_pi5():
    base = BaseController('/dev/ttyAMA0', 115200)
else:
    base = BaseController('/dev/serial0', 115200)

在上面的代码块我们导入并实例化用于底盘控制的库，接下来通过改变云台水平舵机和倾斜舵机的角度来控制云台运动。

改变以下代码中的值，以下代码中：
- input_x 为云台水平舵机的转动角度，水平舵机平面 PAN 轴转动范围 ±180°（总范围360°）；
- input_y 为云台倾斜舵机的转动角度，倾斜舵机竖直平面 TILT 轴转动范围 -45°~90°（总范围135°）；
- input_speed 为云台转动的速度，当速度值为 0 时，转动速度最快；
- input_acc 为云台转动开始和结束时的加速度，数值越小启停越平滑。当加速度值为 0 时，则按照最大的加速度转动。

运行以下代码，可以看见云台会向右且向上分别运动 45° 后停止。

In [ ]:
input_x = 45
input_y = 45
input_speed = 0
input_acc = 0

base.gimbal_ctrl(input_x, input_y, input_speed, input_acc)

除了可以通过改变云台中两个舵机的转动角度来控制云台运动，还可以直接控制云台连续运动。

改变以下代码中的值，以下代码中：

- input_x 为云台水平舵机的转动模式。当值为 -1 时，则表示为云台连续向左转动（顺时针转动）；当值为 1 时，则表示为云台连续向右转动（逆时针转动）；当值为0时，则表示停止转动。
- input_y 为云台倾斜舵机的转动模式。当值为 -1 时，则表示为云台倾斜舵机连续向下转动（低头）；当值为 1 时，则表示为云台倾斜舵机连续向上转动（抬头）；当值为0时，则表示停止转动。
- input_speed 为云台转动的速度。

其中，只有input_x 和 input_y 的值都为 2 时，云台会自动回归中位所处的位置。

运行以下代码，云台会向左运动，直到运动到 180° 时停止。

In [ ]:
input_x = -1
input_y = 0
input_speed = 0

base.gimbal_base_ctrl(input_x, input_y, input_speed)

运行以下代码，云台会向上运动，运动到 90° 时停止。

In [ ]:
input_x = 0
input_y = 1
input_speed = 0

base.gimbal_base_ctrl(input_x, input_y, input_speed)

最后，运行以下代码让云台回归至中位。

In [ ]:
input_x = 2
input_y = 2
input_speed = 0

base.gimbal_base_ctrl(input_x, input_y, input_speed)

## LED灯光控制
### 基础介绍
WAVE ROVER和UGV系列产品，驱动板上集成了两路12V开关控制接口（实际最高电压会随着电池电压而改变），分别为由ESP32的IO4和IO5引脚通过MOS管来控制，每路对应有两个接口，一共4个12V开关控制接口，按照默认的组装方式，IO4控制底盘大灯（WAVE ROVER没有底盘大灯），IO5控制头灯，你可以通过向下位机发送相应的指令来控制这两路接口的开关以及控制电压高低（但由于MOS管控制本身存在一定的延时所以ESP32的IO输出的PWM与实际接口输出的电压不成线性关系）。

对于没有配有LED灯的产品，你可以自行在这两路12V开关控制接口上扩展耐压12.6V的LED灯（一般情况下耐压12V的也可以，因为为了安全和保护电池，产品的UPS不会将电池电量充到12V以上），你也可以在剩余的开关控制接口上扩展其它外设——例如耐压12V的水弹枪波箱，直接连接在IO5控制的接口上，即可实现自动瞄准和射击的功能。

如果需要运行代码块内的程序，你可以选中需要运行的代码块，然后按Ctrl+Enter来运行程序，如果你在手机或平板电脑上使用JupyterLab，你可以按上方的三角形播放按键来运行代码块。

在上面的代码块中我们导入并实例化了用于控制底盘的库，接下来控制IO4接口的开关，变量IO4_PWM为ESP32的IO4引脚输出的PWM，变量范围为0-255，当此变量值为0时，IO4控制的接口开关关闭；当此变量为255时，IO4控制的接口开关输出的电压接近UPS的BAT电压（当前UPS内三节锂电池串联的电压）。

运行以下代码块开启IO4控制的接口开关（亮起底盘大灯）。
注意: WAVE ROVE 没有底盘大灯，所以运行以下代码块是没有变化的，需要运行再下一个开启头灯的代码块才会开启头灯，头灯位于摄像头云台上，如果产品没有配有摄像头云台则没有头灯。

In [ ]:
IO4_PWM = 255
IO5_PWM = 0

base.lights_ctrl(IO4_PWM, IO5_PWM)

运行以下代码块开启IO5控制的接口开关（亮起云台头灯）。
注意：如果产品没有配有摄像头云台则没有头灯。

In [ ]:
IO4_PWM = 255
IO5_PWM = 255

base.lights_ctrl(IO4_PWM, IO5_PWM)

如果你的产品带有LED灯，到目前为止应该已经全部亮起了，你可以运行以下代码块来降低LED灯的亮度。

In [ ]:
IO4_PWM = 64
IO5_PWM = 64

base.lights_ctrl(IO4_PWM, IO5_PWM)

最后，运行以下代码块来关闭LED灯。

In [ ]:
base.lights_ctrl(0, 0)

这里运行一个整合了云台功能的代码块

In [ ]:
import time
base.gimbal_ctrl(0, 0, 0, 0)
base.lights_ctrl(255, 255)
time.sleep(1)
base.gimbal_ctrl(45, 0, 0, 0)
base.lights_ctrl(0, 0)
time.sleep(1)
base.gimbal_ctrl(-45, 0, 0, 0)
base.lights_ctrl(255, 255)
time.sleep(1)
base.gimbal_ctrl(0, 90, 0, 0)
base.lights_ctrl(0, 0)
time.sleep(1)
base.gimbal_ctrl(0, 0, 0, 0)